# Forest Benchmarking STAC Catalog

This notebook demonstrates how to generate a STAC catalog using a sample of the following datasets: 
- [GFLandsat_V1 dataset](https://www.ntsg.umt.edu/project/landsat/landsat-gapfill-reflect.php). The data represents gap-filled montly observations derived from Lansat and MODIS sensors at a 30m spatial resolution generated with the HISTARFM algorithm [(Moreno-Martínez et al. 2020)](https://www.sciencedirect.com/science/article/pii/S0034425720302716).
- LandTrendr is a time series analysis algorithm that uses Landsat imagery to detect changes in land cover and land use over time. The algorithm was implemented in Google Earth Engine by [Kennedy et al. (2018)](https://www.mdpi.com/2072-4292/10/5/691).
- National Agriculture Imagery Program (NAIP). Ortho photography collected by the USDA during the agricultural peak growing season. 
- 3DEP Digital Elevation Model. 

In [1]:
import os
import glob 
import json
from datetime import datetime

import pystac
from pystac.extensions.eo import Band, EOExtension
from pystac.extensions.label import LabelExtension, LabelType, LabelClasses

import ee
import geopandas as gpd
import rasterio

# Generate local repository for each dataset in the catalog

## Catalog file structure

```
root
|___dataset collection
|   |__qq_id_dataset_id
|      |__COG
|      |__Thumbnail
|      |__Metadata
|    
|__label collection
   |__qq_id-labels
      |__FeatureCollection
```

In [2]:
# Initialize the Earth Engine module.
# You need to setup your Google Earth Engine API key before running this cell.
ee.Initialize()

# Build catalog

In [3]:
qq_shp = gpd.read_file('../data/external/usfs_stands/oregon_quarter_quads_sample.geojson')

fbench = pystac.Catalog(
    id = 'forest_benchmarking', 
    description = 'STAC implementation for modelling and benchmarking forest attributes',
    title = 'STAC implementation for modelling and benchmarking forest attributes',
)

In [4]:
def get_qq_bbox(qq_id):
    from shapely import geometry
    
    geom = qq_shp[qq_shp['CELL_ID'] == qq_id].geometry.values[0]
    if isinstance(geom, geometry.MultiPolygon):
        geom = geometry.box(*geom.bounds, ccw=True)

    geom_json = json.loads(gpd.GeoSeries(geom).to_json())
    # json.dumps(geometry.mapping(geom))
    
    return geom_json

In [5]:
def create_label_item(catalog, label_path, label_attr):
    """
    Create a label item.

    Parameters
    ----------
    catalog : pystac.Catalog
        The catalog with the items that will be used to create the label item.
    label_path : str
        The path to the label geojson.
    label_attr : dict
        Dictionary with attributes for the label item.

    Returns
    -------
    label_item : pystac.Item
    """
    EPSG = 'EPSG:4326'

    # Read the label data
    label_data = gpd.read_file(label_path)
    bbox = label_data.total_bounds.tolist()
    label_classes = label_data[label_attr['label_properties'][0]].unique().tolist()
    label_data = label_data.to_crs(EPSG)
    label_data = label_data.to_json()
    label_data = json.loads(label_data)

    # Label id 
    label_id = label_path.split('/')[-1].replace('.geojson', '')

    # Create label item
    label_item = pystac.Item(
        id=f'{label_id}-labels',
        geometry=label_data,
        bbox=bbox,
        datetime=label_attr['label_date'],
        properties={}
    )

    label = LabelExtension.ext(label_item, add_if_missing=True)
    label.apply(
        label_description=label_attr['label_description'],
        label_type=label_attr['label_type'],
        label_properties=label_attr['label_properties'],
        label_tasks=label_attr['label_tasks'],
        label_classes= [
            LabelClasses.create(classes=label_classes, name=label_attr['label_name'])
        ]
    )

    label.add_geojson_labels(href=label_path)

    # Iterate over all items in the catalog and add references to assets in the same stand.
    # We'll need a better way to do this for large collections.
    label_cell_id = label_id.split('_')[0]
    for item in catalog.get_all_items():
        if item.id.split('_')[0] == label_cell_id:
            label.add_source(item, assets=['image'])

    return label_item


def create_item(image_path, thumb_path, metadata_path):
    """
    Create a STAC item.

    Parameters
    ----------
    image_path : str
        Path to the COG image.
    metadata_path : str
        Path to the metadata.

    Returns
    -------
    item : pystac.Item
    """
    # Read the image data
    with rasterio.open(image_path) as src:
        image_data = src.read()
        image_meta = src.profile
        image_meta['crs'] = src.crs.to_epsg()
        image_meta['transform'] = src.transform
        image_meta['bounds'] = list(src.bounds)

    # Read the metadata
    with open(metadata_path) as f:
        metadata = json.load(f)

    # Collect image properties
    image_date = datetime.utcfromtimestamp(metadata['properties']['system:time_start']/1000)
    image_id = image_path.split('/')[-1].replace('-cog.tif', '')
    cell_id = int(image_id.split('_')[0])
    image_geom = get_qq_bbox(cell_id)
    image_height = image_data.shape[1]
    image_width = image_data.shape[-1]
    image_bands = metadata['bands']

    # Create item
    item = pystac.Item(
        id=image_id,
        geometry=image_geom['features'][0]['geometry'],
        bbox=image_geom['bbox'],
        datetime=image_date,
        properties={},
    )

    # Add the EO extension
    bands = [Band.create(name=b['id'], common_name=b.get('name')) for b in image_bands]
    eo = EOExtension.ext(item, add_if_missing=True)
    eo.apply(bands=bands)

    # Add Assets
    github_url = 'https://raw.githubusercontent.com/Ecotrust/forestbenchmarking/main/'
    item.add_asset('image', pystac.Asset(href=github_url + image_path[3:], media_type=pystac.MediaType.COG))
    item.add_asset('metadata', pystac.Asset(href=github_url + metadata_path[3:], media_type=pystac.MediaType.JSON))
    item.add_asset('thumbnail', pystac.Asset(href=github_url + thumb_path[3:], media_type=pystac.MediaType.PNG))

    return item


# Build STAC catalog

In [6]:
# Generate paths to the data
import re


collection_descriptions = {'GFLandsat_V1': 'GFLandsat V1 dataset for Oregon state',
                           'LandTrendr': 'LandTrendr implementation for Google Earth Engine (GEE) platform',
                           'NAIP': 'NAIP imagery for Oregon State',
                           '3DEP': '3DEP elevation data for Oregon State'}

item_paths = {'labels': glob.glob('../data/external/labels/*.geojson'), 'data':{}}

collection_paths = [d[0] for d in os.walk('../data/processed') 
                    if len(list(filter(None, d[0].split('/')))) == 4]

for path in collection_paths:
    collection_name = path.split('/')[-1]
    item_paths['data'][collection_name] = {
        'images': glob.glob(f'{path}/**/*.tif', recursive=True),
        'metadata': glob.glob(f'{path}/**/*.json', recursive=True),
        'thumbnail': glob.glob(f'{path}/**/*.png', recursive=True),
        'description': collection_descriptions[collection_name]
    }

In [7]:
lc_attr = {
    'label_description':'USGS Anderson Level 1 land use code',
    'label_type': LabelType.VECTOR,
    'label_properties': ['USGS_ANDER'],
    'label_name': 'USGS Anderson Level 1 land use code',
    'label_date': datetime(2009,1,1),
    'label_tasks': ['classification']
}

label_collection = pystac.Collection(
    id=f'{fbench.id}-labels',
    description='',
    extent=pystac.SpatialExtent(qq_shp.total_bounds.tolist())
)

for dataset in item_paths['data'].keys():
    # Create one collection for each dataset
    fbench_collection = pystac.Collection(
        id=dataset,
        description=item_paths['data'][dataset]['description'],
        extent=pystac.SpatialExtent(qq_shp.total_bounds.tolist())
    )

    dataset_paths = zip(
        item_paths['data'][dataset]['images'], 
        item_paths['data'][dataset]['metadata'], 
        item_paths['data'][dataset]['thumbnail'] 
    )

    # Create items
    for image_path, metadata_path, thumbnail_path in dataset_paths:
        item = create_item(image_path, thumbnail_path, metadata_path)
        fbench_collection.add_item(item)

    fbench.add_child(fbench_collection)

# Create labels and add references to the source items
for label_path in item_paths['labels']:
    label_item = create_label_item(fbench, label_path, lc_attr)
    label_collection.add_item(label_item)
    
fbench.add_child(label_collection)

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as 

In [8]:
fbench.describe()

* <Catalog id=forest_benchmarking>
    * <Collection id=3DEP>
      * <Item id=108243_3DEP_10mDEM>
      * <Item id=108244_3DEP_10mDEM>
      * <Item id=109086_3DEP_10mDEM>
      * <Item id=109087_3DEP_10mDEM>
      * <Item id=109088_3DEP_10mDEM>
    * <Collection id=GFLandsat_V1>
      * <Item id=108243_Gap_Filled_Landsat_CONUS_2011_leafon>
      * <Item id=108244_Gap_Filled_Landsat_CONUS_2011_leafon>
      * <Item id=109086_Gap_Filled_Landsat_CONUS_2011_leafon>
      * <Item id=109087_Gap_Filled_Landsat_CONUS_2011_leafon>
      * <Item id=109088_Gap_Filled_Landsat_CONUS_2011_leafon>
    * <Collection id=LandTrendr>
      * <Item id=108243_LandTrendr_8B_SWIR1-NBR_2011>
      * <Item id=108244_LandTrendr_8B_SWIR1-NBR_2011>
      * <Item id=109086_LandTrendr_8B_SWIR1-NBR_2011>
      * <Item id=109087_LandTrendr_8B_SWIR1-NBR_2011>
      * <Item id=109088_LandTrendr_8B_SWIR1-NBR_2011>
    * <Collection id=NAIP>
      * <Item id=108243_OR_NAIP_DOQQ_2011>
      * <Item id=108244_OR_NAIP_DOQ

# Validate

In [9]:
fbench.normalize_hrefs('forest_benchmarking')
fbench.validate()

['https://schemas.stacspec.org/v1.0.0/catalog-spec/json-schema/catalog.json']

# Save the catalog

In [10]:
!rm -r ../data/forest_benchmarking_catalog
fbench.save(catalog_type=pystac.CatalogType.SELF_CONTAINED, dest_href='../data/forest_benchmarking_catalog')